In [ ]:
#!pip install -q accelerate==0.21.0 peft==0.4.0 bitsandbytes==0.40.2 transformers==4.31.0 trl==0.4.7
# !pip install --upgrade transformers accelerate bitsandbytes
!pip install accelerate
!pip install peft
!pip install bitsandbytes
!pip install transformers
!pip install datasets

print("successfully installed all the packages")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 20.5 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible

In [ ]:
# Import all the necessary modules
import torch
from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
#   prepare_model_for_int8_training,
    prepare_model_for_kbit_training,
)
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForSeq2Seq, BitsAndBytesConfig

import warnings
warnings.filterwarnings('ignore')



In [ ]:
# Load the dataset from huggingface and get 5000 records for experiment
from datasets import load_dataset

dataset_name = "NumbersStation/NSText2SQL"
dataset=load_dataset(dataset_name, split="train")
dataset = dataset.remove_columns(["source"])

subset_size = min(1000, len(dataset))
dataset = dataset.select(range(subset_size))

train_test_split = dataset.train_test_split(test_size=0.2, seed=42)
train_dataset = train_test_split["train"]
eval_dataset = train_test_split["test"]

print(len(train_dataset))
print(len(eval_dataset))

Generating train split:   0%|          | 0/289288 [00:00<?, ? examples/s]

800
200


In [ ]:
#!nvidia-smi

In [ ]:
# Load Llama Model
#base_model="codellama/CodeLlama-7b-hf"
#base_model="meta-llama/Meta-Llama-3-8B-Instruct"
#base_model="mistralai/Mistral-7B-Instruct-v0.1" Not working GPU RAM issue
base_model="NousResearch/Hermes-2-Pro-Mistral-7B"
#base_model="meta-llama/Llama-2-7b-chat-hf" ## Working
#base_model = "NousResearch/Llama-2-7b-chat-hf"



# For Llama 3 Model
# Define the BitsAndBytesConfig object
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

# quantization_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_compute_dtype=torch.float16,
# #    load_in_8bit_fp32_cpu_offload=True
# )

# # load_in_8bit_fp32_cpu_offload=True,
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     quantization_config=quantization_config,

#     device_map="auto",  # Automatically map the model to the available devices
# )

# Configure the BitsAndBytesConfig for 8 bits quantization
# quant_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_use_double_quant=True,
#     bnb_8bit_quant_type="nf4",  # "fp4" or "nf4" can be used
# )

# Load the model with CPU offloading with 8 bits
device_map = {"": 0}
model = AutoModelForCausalLM.from_pretrained(
    base_model,
    device_map=device_map, # "auto",
    torch_dtype=torch.float16,
    quantization_config=quantization_config,
#    load_in_8bit_fp32_cpu_offload=True  # Offload some modules to CPU
)


# For Llama 2 Model with 4 bits and 8 bits
# model = AutoModelForCausalLM.from_pretrained(
#     base_model,
#     load_in_8bit=True,
#  #   load_in_4bit=True,
#     torch_dtype=torch.float16,
#     device_map= "auto",
# )

tokenizer = AutoTokenizer.from_pretrained(base_model, trust_remote_code=True)

# #Define special tokens for Llama 3
# special_tokens_dict = {'additional_special_tokens': ['<SQL>', '</SQL>', '[INST]', '[/INST]', '<SYS>', '</SYS>']}
# tokenizer.add_special_tokens(special_tokens_dict)
# model.resize_token_embeddings(len(tokenizer))


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.93G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.add_eos_token = True
tokenizer.pad_token_id = 0
tokenizer.padding_side = "left"

In [ ]:
def tokenize(prompt):
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=512,
        padding=False,
        return_tensors=None,
    )

    # "self-supervised learning" means the labels are also the inputs:
    result["labels"] = result["input_ids"].copy()

    return result

In [ ]:
# Convert into promp for dataset
def generate_and_tokenize_prompt(data_point):
    full_prompt =f"""You are a powerful text-to-SQL model.
                     Your job is to answer questions about a database.
                     You are given a question and context regarding one or more tables.

You must output the SQL query that answers the question.

### Input:
{data_point["instruction"]}

### Response:
{data_point["output"]}
"""
    return tokenize(full_prompt)

In [ ]:
tokenized_train_dataset = train_dataset.map(generate_and_tokenize_prompt)
tokenized_val_dataset = eval_dataset.map(generate_and_tokenize_prompt)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [ ]:
#tokenized_train_dataset[0]

In [ ]:
# LoRA Configuration

model.train() # put model back into training mode

model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=128,
    lora_alpha=128,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)
model = get_peft_model(model, config)



In [ ]:
# set this to the adapter_model.bin file want to resume from
resume_from_checkpoint = ""

if resume_from_checkpoint:
    if os.path.exists(resume_from_checkpoint):
        print(f"Restarting from {resume_from_checkpoint}")
        adapters_weights = torch.load(resume_from_checkpoint)
        set_peft_model_state_dict(model, adapters_weights)
    else:
        print(f"Checkpoint {resume_from_checkpoint} not found")

In [ ]:
# Traning parameters
# batch_size = 8
# per_device_train_batch_size = 4
# gradient_accumulation_steps = batch_size // per_device_train_batch_size
# output_dir = "./results"

training_args = TrainingArguments(
        per_device_train_batch_size= 8,
        per_device_eval_batch_size = 8,
        gradient_accumulation_steps= 1,
        # per_device_train_batch_size=per_device_train_batch_size,
        # gradient_accumulation_steps=gradient_accumulation_steps,
        warmup_steps=100,
#       max_steps=400,
        max_steps=-1,
        num_train_epochs= 3,
        learning_rate=1e-5,
        fp16=True,
        optim="paged_adamw_32bit", # "adamw_torch",
        eval_strategy="steps", # if val_set_size > 0 else "no",
        save_strategy="steps",
        logging_steps=25,
        eval_steps=25,
        save_steps=25,
        output_dir="./results",
        # save_total_limit=3,
        load_best_model_at_end=False,
        # ddp_find_unused_parameters=False if ddp else None,
        group_by_length=True, # group sequences of roughly the same length together to speed up training
        # report_to="wandb", # if use_wandb else "none",
        # run_name=f"codellama-{datetime.now().strftime('%Y-%m-%d-%H-%M')}", # if use_wandb else None,
        gradient_checkpointing=True,
       # gradient_checkpointing_kwargs={'use_reentrant':False}
    )


In [ ]:
# Trainer Parameter for training
trainer = Trainer(
    model=model,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_val_dataset,
    args=training_args,
    data_collator=DataCollatorForSeq2Seq(
        tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
    ),
)

In [ ]:
# To stop CUDA overflow
model.config.use_cache = False


In [ ]:
# Traning
import warnings
warnings.filterwarnings('ignore')

trainer.train()

Step,Training Loss,Validation Loss
25,2.021300,1.793757
50,1.625800,1.273403
75,0.957900,0.848975
100,0.784400,0.758850
125,0.682000,0.692332
150,0.625900,0.628774
175,0.587800,0.593702
200,0.576200,0.559386
225,0.499100,0.543446
250,0.493600,0.523248


TrainOutput(global_step=300, training_loss=0.8202882766723633, metrics={'train_runtime': 2234.7001, 'train_samples_per_second': 1.074, 'train_steps_per_second': 0.134, 'total_flos': 3.2229379473408e+16, 'train_loss': 0.8202882766723633, 'epoch': 3.0})

In [ ]:
# Evaluation
eval_results = trainer.evaluate()
print(f"Evaluation Results: {eval_results}")

Evaluation Results: {'eval_loss': 0.5056775808334351, 'eval_runtime': 57.5857, 'eval_samples_per_second': 3.473, 'eval_steps_per_second': 0.434, 'epoch': 3.0}
